In [3]:
import time
import sys
import functools
import numpy as np
import trimesh
from pvtrace import *
import logging
logging.getLogger('trimesh').setLevel(logging.CRITICAL)

In [4]:
world = Node(
    name="world (air)",
    geometry=Sphere(
        radius=50.0,
        material=Material(refractive_index=1.0),
    )
)

In [5]:
box = Node(
    name="box (glass)",
    geometry = Mesh(
        trimesh = trimesh.load('colortest4.gltf').dump().sum(),
        material = Material(
            refractive_index = 1.5,
            components = [
                Absorber(coefficient = 0.525),
                ]
        ),
    ),
    parent=world
)


Unfortunately the trimesh has lost its color properties:

In [335]:
box.geometry.trimesh.visual.vertex_colors

TrackedArray([[102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102, 255],
              [102, 102, 102

But we can create facets by loading the gltf _before_ defining it in the geometry and saving the color data for each face (triangle) into a dataframe.

In [336]:
trimesh2 = trimesh.load('colortest4.gltf')

In [337]:
import pandas as pd
df = pd.DataFrame(columns=['vertices','colors'])
vert = []
colvert = []
for k in range(len(trimesh2.geometry)):
    for j in trimesh2.dump()[k].triangles_center:
        color = trimesh2.dump()[k].visual.to_color().vertex_colors
        vert.append(j)
        colvert.append(color)
df['vertices']=vert
df['colors']=colvert
df

,vertices,colors
0,"[1.0, -0.3333333333333333, -0.33333317438761395]","[5, 0, 163, 255]"
1,"[0.3333331346511841, -0.3333333333333333, 1.00...","[5, 0, 163, 255]"
2,"[-0.9999999006589254, -0.3333333333333333, -0....","[5, 0, 163, 255]"
3,"[0.3333333333333333, 0.3333333333333333, -0.99...","[5, 0, 163, 255]"
4,"[0.9999998013178507, 0.3333333333333333, 0.333...","[5, 0, 163, 255]"
5,"[-0.3333335320154826, 0.3333333333333333, 1.00...","[5, 0, 163, 255]"
6,"[-1.0, 0.3333333333333333, 0.33333317438761395]","[5, 0, 163, 255]"
7,"[-0.3333332339922587, -0.3333333333333333, -1.0]","[5, 0, 163, 255]"
8,"[0.3333331346511841, 1.0, -0.3333328564961751]","[163, 0, 0, 255]"
9,"[-0.3333335320154826, 1.0, 0.3333335518836975]","[163, 0, 0, 255]"


Now when we define the trimesh within the geometry we can cross-reference each triangle's location with the previously defined dataframe to get its color, and then create the facets from there.

In [339]:
all_colors = []
for k in range(len(box.geometry.trimesh.triangles_center)):
    match = next((elem for elem in df['vertices'].values if np.allclose(elem, box.geometry.trimesh.triangles_center[k])), False)
    trueWhere = df[df['vertices'].values.tolist()==match].index.values
    getColor = next(elem for elem in trueWhere if trueWhere.tolist().count(elem)==3)
    color = df['colors'].iloc[getColor].tolist()
    print(color)
    all_colors.append(color)

[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[5, 0, 163, 255]
[163, 0, 0, 255]
[163, 0, 0, 255]
[163, 149, 0, 255]
[163, 149, 0, 255]


In [344]:
from webcolors import rgb_to_name

In [358]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

for requested_colour in all_colors:
    actual_name, closest_name = get_colour_name((requested_colour[0],requested_colour[1], requested_colour[2]))
    print(closest_name)

darkblue
darkblue
darkblue
darkblue
darkblue
darkblue
darkblue
darkblue
darkred
darkred
darkgoldenrod
darkgoldenrod
